<div style='background-color: lightgrey;
    border: 0.5em solid black;
    border-radius: 0.5em;
    padding: 1em;'>
    <h2>Activité 01-XX</h2>
    <h1>La carte de vigilance météorologique</h1>
</div>

In [178]:
import requests

In [197]:
def interroger_API_vigilance_meteo():
    url = """https://public.opendatasoft.com/api/records/1.0/search/?dataset=vigilance-meteorologique&q=&rows=110"""
    reponse = requests.get(url)
    reponse = reponse.json()
    tableau = []
    for elem in reponse['records']:
        dico = {'nom_dep' : elem['fields']['nom_dept'],
                'couleur' : elem['fields']['couleur'],
                'contour' : elem['fields']['geo_shape']}
        if dico['couleur'] != 'Vert':
            dico['risque'] = 'risque ' + elem['fields']['risque_valeur'].lower()
        tableau.append(dico)
    return tableau

In [198]:
donnees = interroger_API_vigilance_meteo()

In [199]:
import folium

In [200]:
COULEURS = {'Vert': lambda x: {'color': '#000000', 'weight': 1, 'fillColor': '#008000', 'fillOpacity': 1},
            'Jaune': lambda x: {'color': '#000000', 'weight': 1, 'fillColor': '#ffff00', 'fillOpacity': 1},
            'Orange': lambda x: {'color': '#000000', 'weight': 1, 'fillColor': '#ff5e13', 'fillOpacity': 1},
            'Rouge': lambda x: {'color': '#000000', 'weight': 1, 'fillColor': '#ff0000', 'fillOpacity': 1}}

In [201]:
def creer_carte_vigilance(donnees):
    carte = folium.Map(location=(46.513 ,2.576), tiles=None, zoom_start=6)
    for elem in donnees:
        couleur = COULEURS[elem['couleur']]
        if elem['couleur'] != 'Vert':
            message = f"{elem['nom_dep']} : {elem['risque']}"
        else:
            message = None
        folium.GeoJson(elem['contour'], tooltip=message, style_function=couleur).add_to(carte)
    return carte

In [202]:
ma_carte = creer_carte_vigilance(donnees)

In [203]:
ma_carte.save('carte_vigilance.html')